In [ ]:
import time  # 시간 지연을 위한 모듈

# 멤버십 정보를 저장할 딕셔너리 (전화번호: 포인트)
membership_dict = {}

# 판매할 메뉴 항목 리스트 (이름, 가격)
menu_list = [
    {'name': '바나나', 'price': 4000},
    {'name': '딸기', 'price': 5000},
    {'name': '벌집꿀', 'price': 7000},
    {'name': '그레놀라', 'price': 7000},
    {'name': '초코쉘', 'price': 7000},
    {'name': '요거트 아이스크림', 'price': 3000}
]

# 유효한 숫자를 입력받기 위한 함수 (메뉴 선택, 수량 선택 등)
def get_valid_number_input(msg, min_no=1, max_no=5):
    try:
        number = int(input(msg))
        if number < min_no:
            print(f"{min_no} 이상 숫자를 입력하세요")
            return None
        elif number > max_no:
            print(f"{max_no} 이하 숫자를 입력하세요")
            return None
        else:
            return number
    except ValueError:
        print("숫자를 입력하세요")
        return None

# 기본 메뉴 화면 출력 함수
def display_menu():
    print('*' * 40)
    print('어서오세요, ‘요렇게’에 오신 것을 환영합니다:)')
    print("Menu\t\t\t\tPrice")
    print("-" * 30)
    print("요거트 아이스크림(250g)\t3,000")
    print("-" * 30)
    print("\nNo. Topping\t\tPrice")
    print("-" * 30)
    for idx, item in enumerate(menu_list, start=1):
        print(f"{idx:<5} {item['name']:<10}\t{item['price']:,}")
    print("-" * 30)
    print("*" * 40)

# 장바구니 상태 출력 함수
def display_cart(cart, total_price):
    print("\n" + "=" * 40)
    print("현재 장바구니")
    for i, item in enumerate(cart, start=1):
        print(f"{i}. {item['name']} x {item['quantity']} = {item['price'] * item['quantity']:,}원")
    print("-" * 40)
    print(f"Total = {total_price:,}원")

# 메뉴를 장바구니에 추가하는 함수
def add_menu(cart, total_price):
    current_total = sum(item['quantity'] for item in cart)
    remaining = 16 - current_total  # 최대 수량 15개 (아이스크림 포함)

    if remaining <= 0:
        print("장바구니에는 최대 15개까지만 담을 수 있습니다.")
        return total_price

    # 토핑 선택
    print("\n메뉴 목록:")
    for i, item in enumerate(menu_list, start=1):
        print(f"{i}. {item['name']} : {item['price']:,}원")
    idx = get_valid_number_input("추가할 토핑 번호를 골라주세요 : ", 1, len(menu_list))
    if idx:
        idx -= 1
        qty = get_valid_number_input(f"수량을 선택해주세요 (남은 수량: {remaining}) : ", 1, remaining)
        if qty:
            selected_item = menu_list[idx]
            # 이미 있는 항목이면 수량만 증가
            for item in cart:
                if item['name'] == selected_item['name']:
                    item['quantity'] += qty
                    if item['quantity'] > 15:
                        item['quantity'] = 15
                    break
            else:
                cart.append({'name': selected_item['name'], 'quantity': qty, 'price': selected_item['price']})
            total_price += selected_item['price'] * qty
    return total_price

# 장바구니에서 메뉴를 삭제하는 함수
def del_menu(cart, total_price):
    display_cart(cart, total_price)
    idx = get_valid_number_input("삭제할 메뉴의 번호를 골라주세요 : ", 1, len(cart))
    if idx:
        idx -= 1
        item = cart[idx]
        print(f"'{item['name']}'의 현재 수량: {item['quantity']}")
        del_qty = get_valid_number_input("몇 개를 삭제하시겠습니까? : ", 1, item['quantity'])
        if del_qty:
            item['quantity'] -= del_qty
            total_price -= item['price'] * del_qty
            print(f"'{item['name']}' {del_qty}개가 삭제되었습니다.")
    return total_price

# 주문 과정 중 선택 기능 안내 함수
def select_pro():
    print("메뉴를 확정하려면 o")
    print("메뉴를 삭제하려면 d")
    print("메뉴를 추가하려면 a")
    print("메뉴를 취소하려면 c")
    return input("알파벳을 입력해주세요 : ").lower().strip()

# 결제 여부 확인 함수
def pay_deci():
    return input("주문이 확정되었습니다. 결제를 하시겠습니까? (y/n): ").lower().strip()

# 결제 처리 및 멤버십 적립 여부 묻기
def pay_screen(total_price, num_order):
    print(f"{total_price:,}원 결제하겠습니다. 카드를 삽입해주세요.")
    print("결제가 완료되었습니다.")
    mem = input("적립 하시겠습니까? (y/n): ").lower().strip()
    if mem.lower() == 'y':
        membership_point_save(total_price)
    print(f"주문번호는 {num_order}번 입니다.")

# 멤버십 적립 처리 함수
def membership_point_save(total_price):
    phone = input("전화번호를 입력해주세요 (- 없이): ").strip()
    points = int(total_price * 0.1)  # 10% 포인트 적립

    if phone in membership_dict:
        membership_dict[phone] += points
        print(f"{phone} 번호에 {points}포인트가 추가되어, 총 {membership_dict[phone]}포인트가 있습니다.")
    else:
        membership_dict[phone] = points
        print(f"{phone} 번호로 {points}포인트가 새로 적립되었습니다.")

# 초기화면으로 돌아가기 전 5초 대기 함수
def end_screen_delay():
    print("초기화면으로 돌아갑니다.")
    for i in range(5, 0, -1):
        print(f"{i} ", end='\r', flush=True)
        time.sleep(1)
    print(" ", end='\r')

# 메인 실행 함수
def main():
    num_order = 1  # 주문 번호 초기화
    while True:
        display_menu()  # 메뉴 출력
        # 기본 아이스크림은 항상 포함
        cart = [{'name': menu_list[5]['name'], 'price': menu_list[5]['price'], 'quantity': 1}]
        total_price = 3000
        while True:
            display_cart(cart, total_price)  # 장바구니 출력
            choice = select_pro()  # 사용자 선택

            if choice == 'o':  # 확정
                break
            elif choice == 'd':  # 삭제
                total_price = del_menu(cart, total_price)
            elif choice == 'a':  # 추가
                total_price = add_menu(cart, total_price)
            elif choice == 'c':  # 취소
                print("주문이 취소됩니다.")
                end_screen_delay()
                break
            elif choice == '99':  # 관리자 종료
                print("관리자 중지입니다.")
                return
            else:
                print("입력이 올바르지 않습니다.")

        if choice in ['c', '99']:
            continue
        if total_price == 0:
            print("장바구니가 비어있습니다.")
            end_screen_delay()
            continue

        # 결제 여부 확인 및 처리
        if pay_deci() == 'y':
            pay_screen(total_price, num_order)
            num_order += 1
        end_screen_delay()

# 프로그램 시작
main()